In [1]:
# Import my packages.
import flymovie as fm
import os
import copy
import scipy
import skimage as ski
import scipy.ndimage as ndi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
import dask
from time import sleep
import seaborn as sns
import itertools

from math import sqrt
from skimage.feature import blob_dog, blob_log, blob_doh
from skimage.color import rgb2gray

from importlib import reload

# Activate package autoload.
%load_ext autoreload
%autoreload 2

In [32]:
from flymovie.cnn_models.siamese_cnn import make_base_cnn, make_embedding, make_siamese_network, SiameseModel
#floof = siamese_network()
base_cnn = make_base_cnn(image_shape=(200,200))
embedding = make_embedding(base_cnn)
siamese_network = make_siamese_network(embedding)
siamese_model = SiameseModel(siamese_network)


ValueError: Input 0 of layer conv1_pad is incompatible with the layer: expected ndim=4, found ndim=6. Full shape received: (None, 1, None, 1, 200, 200)

Let's get fucking focused here. The plan is:

- Generate a training set of triplets of simulated nuclei. Start off-line, just make random negative images and don't worry about curriculum.
- Build the most rudimentary Siamese net and see if it can create an embedding.
- If I can create an embedding, any embedding, this will be a good day.

In [ ]:
# Making 1000 training simulations with duplicates for positives.

outfolder = '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/simulations/sims-202112'
bg_mean_range = [9_000, 11_000]
bg_var_range = [500, 1000]
blob_intensity_mean_range = [10_000, 15_000]
blob_intensity_var_range = [5, 10]
blob_radius_mean_range = [0.5, 1]
blob_radius_var_range = [0.5, 1]
blob_number_range = [20, 200]
num_sims = 2_000
num_replicates=2
z_ij_ratio=2
zdim=20
idim=200
jdim=200
nuc_spacing=1000
nuc_rad=100
fm.make_simulations_from_sampled_params(outfolder=outfolder, bg_mean_range= bg_mean_range, bg_var_range = bg_var_range, blob_intensity_mean_range=blob_intensity_mean_range, 
    blob_intensity_var_range=blob_intensity_var_range, blob_radius_mean_range=blob_radius_mean_range, blob_radius_var_range=blob_radius_var_range, 
    blob_number_range=blob_number_range, num_sims=num_sims, num_replicates=num_replicates, z_ij_ratio=z_ij_ratio, zdim=zdim, idim=idim, jdim=jdim, 
    nuc_spacing=nuc_spacing, nuc_rad=nuc_rad)

OK I'm going to try to set up a siamese network with triplet loss function. Basing it on https://keras.io/examples/vision/siamese_network/

Hold onto your butts

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet

target_shape = (200, 200)

In [ ]:
def preprocess_image(filename):
    """
    Load the specified pkl file, make max intensity projection, normalize.
    """
    image = fm.load_pickle(filename)
    mip = image.max(axis=0)
    mip = mip.astype('float32')
    mip = np.expand_dims(mip, axis=0)
    mip = np.vstack([mip, mip, mip])
    mip = np.swapaxes(mip, 0, 2)
    # Normalize 0-1.
    mip = (mip - np.min(mip)) / np.max(mip) 
    return mip

In [ ]:
## Make lists of anchor, positive, and negative datasets.

# Set directories.
cache_dir = Path('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/simulations/sims-202112')
anchor_images_path = cache_dir / "left"
positive_images_path = cache_dir / "right"

# Create lists of sorted files for anchor and positive images.
anchor_image_files = sorted(
    [str(anchor_images_path / f) for f in os.listdir(anchor_images_path)]
)

positive_image_files = sorted(
    [str(positive_images_path / f) for f in os.listdir(positive_images_path)]
)

image_count = len(anchor_image_files)

# To generate the list of negative images, randomize the list of
# available images and concatenate them together.
negative_image_files = anchor_image_files + positive_image_files
np.random.RandomState(seed=32).shuffle(negative_image_files)
np.random.RandomState(seed=16).shuffle(negative_image_files)
np.random.RandomState(seed=8).shuffle(negative_image_files)

# Apply preprocessing to all input images, return as lists.
anchor_images = list(map(preprocess_image, anchor_image_files))
positive_images = list(map(preprocess_image, positive_image_files))
negative_images = list(map(preprocess_image, negative_image_files))[:1000]

# Convert lists to tf datasets, shuffle the negatives again for good measure.
anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_images)
positive_dataset = tf.data.Dataset.from_tensor_slices(positive_images)
negative_dataset = tf.data.Dataset.from_tensor_slices(negative_images)
negative_dataset = negative_dataset.shuffle(buffer_size=4096)

# Zip three datasets together to make final dataset, where each entry is a triplet of anchor, positive, and negative images.
dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))

# Split dataset into training and validation sets.
train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

# Divide training and validation datasets into batches of size 32, prefetch them 
# (which I still don't totally understand but seems to pre-activate them in some
# meaningrul way).
train_dataset = train_dataset.batch(32, drop_remainder=False)
train_dataset = train_dataset.prefetch(8)

val_dataset = val_dataset.batch(32, drop_remainder=False)
val_dataset = val_dataset.prefetch(8)

In [ ]:

def visualize(anchor, positive, negative):
    """Visualize a few triplets from the supplied batches."""

    def show(ax, image):
        ax.imshow(image)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    fig = plt.figure(figsize=(9, 9))

    axs = fig.subplots(3, 3)
    for i in range(3):
        show(axs[i, 0], anchor[i])
        show(axs[i, 1], positive[i])
        show(axs[i, 2], negative[i])


visualize(*list(train_dataset.take(1).as_numpy_iterator())[0])

In [ ]:
base_cnn = resnet.ResNet50(
    weights="imagenet", input_shape=target_shape + (1,), include_top=False
)

flatten = layers.Flatten()(base_cnn.output)
dense1 = layers.Dense(512, activation="relu")(flatten)
dense1 = layers.BatchNormalization()(dense1)
dense2 = layers.Dense(256, activation="relu")(dense1)
dense2 = layers.BatchNormalization()(dense2)
output = layers.Dense(256)(dense2)

embedding = Model(base_cnn.input, output, name="Embedding")

trainable = False
for layer in base_cnn.layers:
    if layer.name == "conv5_block1_out":
        trainable = True
    layer.trainable = trainable

In [ ]:
class DistanceLayer(layers.Layer):
    """
    This layer is responsible for computing the distance between the anchor
    embedding and the positive embedding, and the anchor embedding and the
    negative embedding.
    """

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, anchor, positive, negative):
        ap_distance = tf.reduce_sum(tf.square(anchor - positive), -1)
        an_distance = tf.reduce_sum(tf.square(anchor - negative), -1)
        return (ap_distance, an_distance)

In [ ]:
anchor_input = layers.Input(name="anchor", shape=target_shape + (3,))
positive_input = layers.Input(name="positive", shape=target_shape + (3,))
negative_input = layers.Input(name="negative", shape=target_shape + (3,))

In [ ]:
distances = DistanceLayer()(
    embedding(resnet.preprocess_input(anchor_input)),
    embedding(resnet.preprocess_input(positive_input)),
    embedding(resnet.preprocess_input(negative_input)),
)

In [ ]:
siamese_network = Model(
    inputs=[anchor_input, positive_input, negative_input], outputs=distances
)

In [ ]:
class SiameseModel(Model):
    """The Siamese Network model with a custom training and testing loops.

    Computes the triplet loss using the three embeddings produced by the
    Siamese Network.

    The triplet loss is defined as:
       L(A, P, N) = max(‖f(A) - f(P)‖² - ‖f(A) - f(N)‖² + margin, 0)
    """

    def __init__(self, siamese_network, margin=0.5):
        super(SiameseModel, self).__init__()
        self.siamese_network = siamese_network
        self.margin = margin
        self.loss_tracker = metrics.Mean(name="loss")

    def call(self, inputs):
        return self.siamese_network(inputs)

    def train_step(self, data):
        # GradientTape is a context manager that records every operation that
        # you do inside. We are using it here to compute the loss so we can get
        # the gradients and apply them using the optimizer specified in
        # `compile()`.
        with tf.GradientTape() as tape:
            loss = self._compute_loss(data)

        # Storing the gradients of the loss function with respect to the
        # weights/parameters.
        gradients = tape.gradient(loss, self.siamese_network.trainable_weights)

        # Applying the gradients on the model using the specified optimizer
        self.optimizer.apply_gradients(
            zip(gradients, self.siamese_network.trainable_weights)
        )

        # Let's update and return the training loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self._compute_loss(data)

        # Let's update and return the loss metric.
        self.loss_tracker.update_state(loss)
        return {"loss": self.loss_tracker.result()}

    def _compute_loss(self, data):
        # The output of the network is a tuple containing the distances
        # between the anchor and the positive example, and the anchor and
        # the negative example.
        ap_distance, an_distance = self.siamese_network(data)

        # Computing the Triplet Loss by subtracting both distances and
        # making sure we don't get a negative value.
        loss = ap_distance - an_distance
        loss = tf.maximum(loss + self.margin, 0.0)
        return loss

    @property
    def metrics(self):
        # We need to list our metrics here so the `reset_states()` can be
        # called automatically.
        return [self.loss_tracker]

In [ ]:
siamese_model = SiameseModel(siamese_network)
siamese_model.compile(optimizer=optimizers.Adam(0.0001))
siamese_model.fit(train_dataset, epochs=1, validation_data=val_dataset)

In [ ]:
sample = next(iter(train_dataset))
visualize(*sample)

anchor, positive, negative = sample
anchor_embedding, positive_embedding, negative_embedding = (
    embedding(resnet.preprocess_input(anchor)),
    embedding(resnet.preprocess_input(positive)),
    embedding(resnet.preprocess_input(negative)),
)

In [ ]:
cosine_similarity = metrics.CosineSimilarity()

positive_similarity = cosine_similarity(anchor_embedding, positive_embedding)
print("Positive similarity:", positive_similarity.numpy())

negative_similarity = cosine_similarity(anchor_embedding, negative_embedding)
print("Negative similarity", negative_similarity.numpy())

OK I'm going to try to build something like the resent50 network myself. Uggh

In [ ]:
a = np.ones((100,100))
fm.save_pickle(a, '/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/simulations/sims-202112/test.pkl')

In [ ]:
np.savetxt('/Users/michaelstadler/Bioinformatics/Projects/rpb1/data/simulations/sims-202112/test.txt', a)